# Fine Tuning a PyTorch Model on Our Fruits Dataset
This notebook will demonstrate how to use PyTorch to fine tune a pre-existing object detection model for our fruits data set.

We will start by loading a pre-trained Faster RCNN model using the PyTorch library and modify it so that it can classify the various fruits in our data set.

Afterwards, we will write our training, evaluation, and testing loops used for training our model.
Our custom Dataset class will be used in combination with a DataLoader for batching our data for training.
During training, we will set it up so that if training is interrupted it can be resumed from where it stopped.
We will see how to save our "best" model found during our training loop.
Once training is completed, we will use our test data set to evaluate the precision, recall, and accuracy of our model.

Finally, we will discuss next steps, including making our model work with the PyTorch lightning framework.